# Grad-CAM Explainability Notebook

## Imports and Setup

In [21]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

## Load models

In [22]:
models = {
  "Custom CNN": tf.keras.models.load_model('/content/models/custom_cnn.keras'),
  "MobileNetV2 (Frozen)": tf.keras.models.load_model('/content/models/malaria_mobilenetv2_frozen.keras'),
  "MobileNetV2 (Fine-tuned)": tf.keras.models.load_model('/content/models/malaria_mobilenetv2_frozen.keras')
}

## Identify the Last Convolutional Layer of Each Model

In [52]:
for name, model in models.items():
    print(f"{name} input shape: {model.input_shape}")
    print(f"{name} last conv: {model_configs[name]['last_conv']}")


Custom CNN input shape: (None, 128, 128, 3)
Custom CNN last conv: conv2d_1
MobileNetV2 (Frozen) input shape: (None, 128, 128, 3)
MobileNetV2 (Frozen) last conv: Conv_1
MobileNetV2 (Fine-tuned) input shape: (None, 128, 128, 3)
MobileNetV2 (Fine-tuned) last conv: Conv_1


## Model Configuration Dictionary

In [24]:
model_configs = {
    "Custom CNN": {
        "input_size": (128, 128),
        "preprocess": lambda x: x / 255.0,
        "last_conv": "conv2d_1"
    },
    "MobileNetV2 (Frozen)": {
        "input_size": (128, 128),
        "preprocess": tf.keras.applications.mobilenet_v2.preprocess_input,
        "last_conv": "Conv_1"
    },
    "MobileNetV2 (Fine-tuned)": {
        "input_size": (128, 128),
        "preprocess": tf.keras.applications.mobilenet_v2.preprocess_input,
        "last_conv": "Conv_1"
    }
}

In [65]:
for model_name, model in models.items():
    print(f"\n{'='*20} Checking Model: {model_name} {'='*20}")

    # Print the last 10 layers to see names and types
    for layer in model.layers[-10:]:
        print(f"Layer Name: {layer.name} | Type: {type(layer)}")

    # Diagnostic: Check if the config name actually exists in this model
    expected_layer = model_configs[model_name]["last_conv"]
    try:
        model.get_layer(expected_layer)
        print(f"✅ Success: '{expected_layer}' found in {model_name}.")
    except ValueError:
        print(f"❌ ERROR: '{expected_layer}' NOT found in {model_name}.")


==================== Checking Model: Custom CNN ====================
Layer Name: conv2d | Type: <class 'keras.src.layers.convolutional.conv2d.Conv2D'>
Layer Name: max_pooling2d | Type: <class 'keras.src.layers.pooling.max_pooling2d.MaxPooling2D'>
Layer Name: conv2d_1 | Type: <class 'keras.src.layers.convolutional.conv2d.Conv2D'>
Layer Name: max_pooling2d_1 | Type: <class 'keras.src.layers.pooling.max_pooling2d.MaxPooling2D'>
Layer Name: flatten | Type: <class 'keras.src.layers.reshaping.flatten.Flatten'>
Layer Name: dense | Type: <class 'keras.src.layers.core.dense.Dense'>
Layer Name: dropout | Type: <class 'keras.src.layers.regularization.dropout.Dropout'>
Layer Name: dense_1 | Type: <class 'keras.src.layers.core.dense.Dense'>
✅ Success: 'conv2d_1' found in Custom CNN.

==================== Checking Model: MobileNetV2 (Frozen) ====================
Layer Name: block_16_depthwise_relu | Type: <class 'keras.src.layers.activations.relu.ReLU'>
Layer Name: block_16_project | Type: <class '

In [72]:
# Test if gradients can even flow through the model
test_img = tf.convert_to_tensor(img_array, dtype=tf.float32)
with tf.GradientTape() as tape:
    tape.watch(test_img)
    p = models['Custom CNN'](test_img)
g = tape.gradient(p, test_img)
print("Gradient check:", g is not None)

Gradient check: True


In [82]:
# Convert Sequential to Functional to ensure gradient connectivity
for name in models:
    m = models[name]
    models[name] = tf.keras.models.Model(inputs=m.inputs, outputs=m.outputs)

## Universal Grad-CAM Function

In [86]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    last_conv_layer = model.get_layer(last_conv_layer_name)

    grad_model = tf.keras.models.Model(
        inputs=model.inputs,
        outputs=[last_conv_layer.output, model.output]
    )

    with tf.GradientTape() as tape:
        img_tensor = tf.cast(img_array, tf.float32)
        # Unpack the list returned by grad_model
        conv_outputs, predictions = grad_model(img_tensor, training=False)

        # Check if predictions is a list (sometimes happens in Sequential/Nested)
        if isinstance(predictions, list):
            predictions = predictions[0]

        # Now .shape will work
        if predictions.shape[-1] == 1:
            class_channel = predictions[:, 0]
        else:
            if pred_index is None:
                pred_index = tf.argmax(predictions[0])
            class_channel = predictions[:, pred_index]

    # Calculate gradients
    grads = tape.gradient(class_channel, conv_outputs)

    if grads is None:
        # Final fallback for stubborn layers: watch the tensor directly
        with tf.GradientTape() as tape:
            conv_outputs, predictions = grad_model(img_tensor, training=False)
            if isinstance(predictions, list): predictions = predictions[0]
            tape.watch(conv_outputs)
            class_channel = predictions[:, 0] if predictions.shape[-1] == 1 else predictions[:, pred_index]
        grads = tape.gradient(class_channel, conv_outputs)

    # Global average pooling and heatmap math
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / (tf.math.reduce_max(heatmap) + 1e-8)
    return heatmap.numpy()

## Image Loading and Preprocessing

In [87]:
def load_and_preprocess_image(img_path, config):
  img = load_img(img_path, target_size=config["input_size"])
  img_array = img_to_array(img)
  img_array = np.expand_dims(img_array, axis=0)
  img_array = config["preprocess"](img_array)
  return img_array, np.array(img)

## Run Grad-CAM For All Models

In [88]:
img_path = "/content/sample_cell.png"
results = {}

for model_name, model in models.items():
  config = model_configs[model_name]
  img_array, orig_img = load_and_preprocess_image(img_path, config)

  preds = model.predict(img_array)

  # If the model outputs 1 neuron with sigmoid:
  if preds.shape[1] == 1:
      pred_class = 0 if preds[0][0] < 0.5 else 1
  else:
      pred_class = np.argmax(preds[0])


  heatmap = make_gradcam_heatmap(
      img_array,
      model,
      last_conv_layer_name=config["last_conv"],
      pred_index=pred_class
  )

  heatmap = cv2.resize(heatmap, config["input_size"])
  heatmap = np.uint8(255 * heatmap)
  heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

  superimposed = cv2.addWeighted(orig_img, 0.6, heatmap, 0.4, 0)

  results[model_name] = superimposed

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step


ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.

## Add Grad-CAM Function

In [53]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
  # Ensure the model is built by calling it with input data
  _ = model(img_array)

  grad_model = tf.keras.models.Model(
      [model.input], # Changed from model.inputs to model.input
      [model.get_layer(last_conv_layer_name).output, model.output]
  )

  with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(img_array)
    if pred_index is None:
      pred_index = tf.argmax(predictions[0])

    if predictions.shape[1] == 1:
      class_channel = predictions[:, 0]  # single neuron output
    else:
      class_channel = predictions[:, pred_index]


  grads = tape.gradient(class_channel, conv_outputs)
  pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

  conv_outputs = conv_outputs[0]
  heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
  heatmap = tf.squeeze(heatmap)

  heatmap = tf.maximum(heatmap, 0) # Corrected typo: tf.maxium -> tf.maximum
  heatmap /= tf.math.reduce_max(heatmap) + 1e-8
  return heatmap.numpy()

## Load and Preprocess Test Images

In [ ]:
img_path = "/content/sample_cell.png"

img = load_img(img_path, target_size=(128, 128))
img_array = img_to_array(img)
img_array = img_array / 255.0
img_array = np.expand_dims(img_array, axis=0)


## Generate and Overlay Grad-CAM

import matplotlib.pyplot as plt

heatmap = make_gradcam_heatmap(
  img_array,
  model,
  last_conv_layer_name="Conv_1"